In [ ]:
import os
import json
from IPython.display import display, clear_output
import ipywidgets as widgets
from datetime import datetime, timedelta # 마감일 비교 및 형식 변환을 위해 datetime 모듈 사용

FILENAME = "tasks_v3.txt"
tasks = []

# ---- 위젯 정의 ----
out = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '10px', 'width': 'auto'})
ui_title = widgets.HTML("<h2>Visual Task Planner 📝</h2>")
load_message_display = widgets.HTML()
stats_display = widgets.HTML()

# --- 1. 할 일 추가 섹션 위젯들 ---
add_task_description_input = widgets.Text(description="할 일 내용:", placeholder='예: 느낌 좋은 카페 탐방', layout={'width': 'auto'})

# 마감일 입력 (Text + ToggleButton + DatePicker)
add_task_due_date_text = widgets.Text(
    description="마감일:",
    placeholder="YYYY-MM-DD 또는 달력 사용",
    layout={'flex': '1 1 auto', 'width': 'auto'} # HBox 내에서 유연하게 너비 조절
)
add_task_calendar_toggle = widgets.ToggleButton(
    value=False, description="달력 열기", icon='calendar', tooltip='마감일을 달력에서 선택',
    layout={'width': '120px'}
)
add_task_datepicker = widgets.DatePicker(
    description='', disabled=False, layout={'display': 'none', 'margin':'5px 0 0 0px'} # 초기 숨김
)
add_due_date_line = widgets.HBox([add_task_due_date_text, add_task_calendar_toggle], layout={'align_items':'center'})
add_due_date_control_area = widgets.VBox([add_due_date_line, add_task_datepicker])


add_task_priority_dropdown = widgets.Dropdown(options=['높음', '중간', '낮음'], value='중간', description='우선순위:', layout={'width': 'auto'})
add_task_button = widgets.Button(description="할 일 추가", button_style='success', icon='plus')

# --- 2. 할 일 완료 표시 섹션 ---
complete_task_number_input = widgets.Text(description="완료할 번호:", placeholder='예: 1', layout={'width': '200px'})
complete_task_button = widgets.Button(description="완료 표시", button_style='warning', icon='check')

# --- 3. 완료 표시 취소 섹션 ---
undo_complete_task_number_input = widgets.Text(description="취소할 번호:", placeholder='예: 1', layout={'width': '200px'})
undo_complete_task_button = widgets.Button(description="완료 취소", icon='undo')

# --- 4. 할 일 정보 수정 섹션 위젯들 ---
update_task_number_input = widgets.Text(description="수정할 번호:", placeholder='예: 1', layout={'width': 'auto', 'margin_bottom':'5px'}) # 번호 입력은 항상 위에
update_task_description_input = widgets.Text(description="새 내용:", placeholder='수정할 경우 입력', layout={'width': 'auto'})

# 마감일 수정 (Text + ToggleButton + DatePicker)
update_task_due_date_text = widgets.Text(
    description="새 마감일:",
    placeholder="YYYY-MM-DD 또는 달력 사용",
    layout={'flex': '1 1 auto', 'width': 'auto'}
)
update_task_calendar_toggle = widgets.ToggleButton(
    value=False, description="달력 열기", icon='calendar', tooltip='새 마감일을 달력에서 선택',
    layout={'width': '120px'}
)
update_task_datepicker = widgets.DatePicker(
    description='', disabled=False, layout={'display': 'none', 'margin':'5px 0 0 0px'} # 초기 숨김
)
update_due_date_line = widgets.HBox([update_task_due_date_text, update_task_calendar_toggle], layout={'align_items':'center'})
update_due_date_control_area = widgets.VBox([update_due_date_line, update_task_datepicker])


update_task_priority_dropdown = widgets.Dropdown(options=['높음', '중간', '낮음'], value='중간', description='새 우선순위:', layout={'width': 'auto'})
update_task_status_dropdown = widgets.Dropdown(options=['미시작', '진행중', '완료'], value='미시작', description='상태:', layout={'width': 'auto'})
update_task_progress_slider = widgets.IntSlider(value=0, min=0, max=100, step=5, description='진행률(%):', continuous_update=False, style={'description_width': 'initial'}, layout={'width': 'auto'})
update_task_button = widgets.Button(description="선택 항목 업데이트", button_style='primary', icon='edit')

# --- 5. 선택한 할 일 삭제 섹션 ---
delete_task_number_input = widgets.Text(description="삭제할 번호:", placeholder='예: 1', layout={'width': '200px'})
delete_task_button = widgets.Button(description="선택한 할 일 삭제", button_style='danger', icon='trash')


# ---- 핵심 로직 함수들 (parse_date, get_due_date_status, get_priority_emoji 등은 이전과 동일) ----
def parse_date(date_str):
    if not date_str: return None
    try: return datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError: return None

def get_due_date_status(due_date_str):
    if not due_date_str: return ""
    due_date = parse_date(due_date_str)
    if not due_date: return " (잘못된 날짜 형식)"
    today = datetime.now().date()
    time_diff = (due_date - today).days
    if time_diff < 0: return "❗(기한 지남)"
    elif time_diff == 0: return "⏰(오늘 마감!)"
    elif time_diff <= 3: return f"⚠️({time_diff}일 후 마감)"
    return ""

def get_priority_emoji(priority_str):
    if priority_str == "높음": return "🔥"
    if priority_str == "중간": return "🟡"
    if priority_str == "낮음": return "🟢"
    return ""

def load_tasks_core():
    global tasks
    tasks = []
    if os.path.exists(FILENAME):
        try:
            with open(FILENAME, 'r', encoding='utf-8') as file:
                for line in file:
                    if line.strip(): tasks.append(json.loads(line.strip()))
            if tasks: return f"'{FILENAME}'에서 할 일을 성공적으로 불러왔습니다. (총 {len(tasks)}개)"
            else: return f"'{FILENAME}' 파일은 비어있지만, 내용은 없습니다. 새 할 일 목록으로 시작합니다."
        except json.JSONDecodeError: return f"오류: '{FILENAME}' 파일의 내용이 올바른 JSON 형식이 아닙니다."
        except Exception as e: return f"할 일을 불러오는 중 오류가 발생했습니다: {e}"
    else: return ""

def save_tasks_core():
    try:
        with open(FILENAME, 'w', encoding='utf-8') as file:
            for task in tasks: file.write(json.dumps(task, ensure_ascii=False) + '\n')
    except Exception as e:
        with out: print(f"⚠️ 파일 저장 중 오류 발생: {e}")

def add_task_logic(description, due_date_str, priority): # due_date_str은 Text 위젯에서 옴
    description_stripped = description.strip()
    if not description_stripped: return "⚠️ 할 일 내용이 비어있습니다."

    # DatePicker에서 None이 올 경우를 대비해, parse_date는 빈 문자열을 None으로 처리함
    # Text 위젯의 값이 ""일 경우 parsed_due_date는 None이 됨
    parsed_due_date = parse_date(due_date_str)
    if due_date_str and not parsed_due_date:
        return "⚠️ 마감일 형식이 잘못되었습니다. (YYYY-MM-DD)"

    new_task = {
        "description": description_stripped, "status": "미시작", "progress": 0,
        "due_date": due_date_str if parsed_due_date or not due_date_str else "", # 유효하거나 빈 문자열일 때만 저장
        "priority": priority
    }
    tasks.append(new_task)
    save_tasks_core()
    return f"✅ '{new_task['description']}' 할 일이 추가되었습니다."

def update_task_logic(task_number_str, new_desc, new_due_date_str, new_priority, new_status, new_progress):
    try:
        task_number = int(task_number_str)
        if not (1 <= task_number <= len(tasks)):
            return f"⚠️ 잘못된 번호입니다. 1부터 {len(tasks)} 사이의 번호를 입력해주세요."

        idx = task_number - 1

        if new_desc.strip(): tasks[idx]['description'] = new_desc.strip()

        # 새 마감일 처리: 빈 문자열이면 삭제, 아니면 형식 검사 후 업데이트
        if new_due_date_str == "":
            tasks[idx]['due_date'] = ""
        elif new_due_date_str: # 값이 있을 때만 파싱 시도
            parsed_due_date = parse_date(new_due_date_str)
            if not parsed_due_date:
                return "⚠️ 새 마감일 형식이 잘못되었습니다. (YYYY-MM-DD)"
            tasks[idx]['due_date'] = new_due_date_str
        # new_due_date_str이 None이거나 (DatePicker에서 초기값)
        # 위젯에 아무것도 입력 안해서 ""가 아닐 경우는 기존 날짜 유지 (Text 위젯은 ""가 됨)

        tasks[idx]['priority'] = new_priority

        actual_progress = new_progress
        current_status = new_status
        if current_status == "완료": actual_progress = 100
        elif current_status == "미시작": actual_progress = 0
        else:
            if actual_progress == 100: current_status = "완료"
            elif actual_progress == 0: current_status = "미시작"
        tasks[idx]['status'] = current_status; tasks[idx]['progress'] = actual_progress

        save_tasks_core()
        return (f"🛠️ '{tasks[idx]['description']}' 할 일 정보 업데이트 완료.")
    except ValueError: return "⚠️ 잘못된 입력입니다. 할 일 번호를 숫자로 정확히 입력해주세요."

# (complete_task_logic, undo_complete_task_logic, delete_task_logic 은 이전과 동일)
def complete_task_logic(task_number_str):
    try:
        task_number = int(task_number_str)
        if 1 <= task_number <= len(tasks):
            idx = task_number - 1
            if tasks[idx]['status'] != "완료":
                tasks[idx]['status'] = "완료"; tasks[idx]['progress'] = 100
                save_tasks_core()
                return f"🎉 '{tasks[idx]['description']}' 할 일을 완료로 표시했습니다."
            else: return "ℹ️ 이미 완료된 할 일입니다."
        else: return f"⚠️ 잘못된 번호입니다. 1부터 {len(tasks)} 사이의 번호를 입력해주세요."
    except ValueError: return "⚠️ 잘못된 입력입니다. 숫자로 번호를 입력해주세요."

def undo_complete_task_logic(task_number_str):
    try:
        task_number = int(task_number_str)
        if 1 <= task_number <= len(tasks):
            idx = task_number - 1
            if tasks[idx]['status'] == "완료":
                tasks[idx]['status'] = "미시작"; tasks[idx]['progress'] = 0
                save_tasks_core()
                return f"🔄 '{tasks[idx]['description']}' 할 일의 완료 표시를 취소했습니다."
            else: return "ℹ️ 완료되지 않은 할 일입니다."
        else: return f"⚠️ 잘못된 번호입니다. 1부터 {len(tasks)} 사이의 번호를 입력해주세요."
    except ValueError: return "⚠️ 잘못된 입력입니다. 숫자로 번호를 입력해주세요."

def delete_task_logic(task_number_str):
    try:
        task_number = int(task_number_str)
        if 1 <= task_number <= len(tasks):
            idx = task_number - 1
            desc = tasks[idx]['description']
            tasks.pop(idx)
            save_tasks_core()
            return f"🗑️ '{desc}' 할 일이 삭제되었습니다."
        else: return f"⚠️ 잘못된 번호입니다. 1부터 {len(tasks)} 사이의 번호를 입력해주세요."
    except ValueError: return "⚠️ 잘못된 입력입니다. 숫자로 번호를 입력해주세요."


def create_task_list_ui():
    if not tasks:
        return widgets.HTML("<p style='color: #555; font-style: italic;'>ℹ️ 등록된 할 일이 없습니다.</p>")
    task_item_widgets = []
    sorted_tasks = sorted(tasks, key=lambda x: (parse_date(x.get('due_date')) is None, parse_date(x.get('due_date'))))
    for index, task in enumerate(sorted_tasks):
        status_emoji = ""; progress_bar_style = ''
        due_date_str = task.get('due_date', ''); priority_str = task.get('priority', '중간')
        if task['status'] == "완료": status_emoji = "🎉"; progress_bar_style = 'success'
        elif task['status'] == "진행중": status_emoji = "⏳"; progress_bar_style = 'info'
        else: status_emoji = "📋"; progress_bar_style = 'warning'
        due_date_info = f" (마감: {due_date_str if due_date_str else '미지정'})"; due_status_emoji = get_due_date_status(due_date_str)
        priority_emoji = get_priority_emoji(priority_str)
        task_description_html = widgets.HTML(value=(f"<div style='margin-bottom: 2px; width: 100%; word-wrap: break-word; overflow-wrap: break-word;'><b>{index + 1}. {priority_emoji}{task['description']}</b> {status_emoji} {due_status_emoji}<br><small style='color:grey;'>   (상태: {task['status']}{due_date_info} | 우선순위: {priority_str})</small></div>"))
        progress_bar = widgets.IntProgress(value=task['progress'], min=0, max=100, step=1, bar_style=progress_bar_style, layout=widgets.Layout(width='auto', flex='1 1 auto', margin='0 5px 0 0'))
        progress_text_label = widgets.Label(value=f"{task['progress']}%", layout=widgets.Layout(width='45px'))
        progress_line_ui = widgets.HBox([progress_bar, progress_text_label], layout=widgets.Layout(align_items='center', width='98%', height='20px'))
        task_item_ui = widgets.VBox([task_description_html, progress_line_ui], layout={'margin': '6px 0px', 'padding': '8px', 'border': '1px solid #e0e0e0', 'border_radius': '5px', 'width': 'auto'})
        task_item_widgets.append(task_item_ui)
    return widgets.VBox(task_item_widgets, layout={'width': '100%'})

def update_stats_display():
    total_tasks = len(tasks); completed_tasks = len([t for t in tasks if t['status'] == '완료'])
    high_priority_tasks = len([t for t in tasks if t.get('priority') == '높음' and t['status'] != '완료'])
    today = datetime.now().date(); urgent_tasks_count = 0
    for task in tasks:
        if task['status'] != '완료':
            due_date = parse_date(task.get('due_date', ''));
            if due_date and (due_date - today).days <= 0: urgent_tasks_count += 1
    stats_html = (f"<div style='padding: 10px; margin-bottom:10px; border: 1px solid #add8e6; background-color: #f0f8ff; border-radius: 5px; color: #333;'>📊 <strong style='color: #00579c;'>간단 통계:</strong> 총 할 일: <span style='font-weight:bold; color: #111;'>{total_tasks}</span>개 | 완료: <span style='font-weight:bold; color:green;'>{completed_tasks}</span>개 | 미완료 '높음': <span style='font-weight:bold; color:red;'>{high_priority_tasks}</span>개 | 오늘/기한만료 미완료: <span style='font-weight:bold; color:darkorange;'>{urgent_tasks_count}</span>개</div>")
    stats_display.value = stats_html

# ---- 버튼 및 DatePicker 이벤트 핸들러 함수들 ----
def display_results(confirmation_message):
    if confirmation_message: print(confirmation_message)
    update_stats_display()
    task_list_display_ui = create_task_list_ui()
    display(task_list_display_ui)

# --- '새 할 일 추가' 섹션의 달력 관련 핸들러 ---
def on_add_calendar_toggle_changed(change):
    if change['new']: add_task_datepicker.layout.display = 'block'; add_task_calendar_toggle.description = "달력 닫기"
    else: add_task_datepicker.layout.display = 'none'; add_task_calendar_toggle.description = "달력 열기"
add_task_calendar_toggle.observe(on_add_calendar_toggle_changed, names='value')

def on_add_date_picked(change):
    if change['new']: add_task_due_date_text.value = change['new'].strftime('%Y-%m-%d')
    else: add_task_due_date_text.value = ""
add_task_datepicker.observe(on_add_date_picked, names='value')

# --- '할 일 정보 수정' 섹션의 달력 관련 핸들러 ---
def on_update_calendar_toggle_changed(change):
    if change['new']: update_task_datepicker.layout.display = 'block'; update_task_calendar_toggle.description = "달력 닫기"
    else: update_task_datepicker.layout.display = 'none'; update_task_calendar_toggle.description = "달력 열기"
update_task_calendar_toggle.observe(on_update_calendar_toggle_changed, names='value')

def on_update_date_picked(change):
    if change['new']: update_task_due_date_text.value = change['new'].strftime('%Y-%m-%d')
    else: update_task_due_date_text.value = "" # DatePicker에서 날짜 선택 취소 시 텍스트 필드 비움
update_task_datepicker.observe(on_update_date_picked, names='value')


def on_add_task_button_clicked(b):
    with out:
        clear_output(wait=True)
        confirmation_message = add_task_logic(
            add_task_description_input.value,
            add_task_due_date_text.value, # DatePicker와 연결된 Text 위젯의 값 사용
            add_task_priority_dropdown.value
        )
        add_task_description_input.value = ''
        add_task_due_date_text.value = '' # 마감일 Text 위젯 초기화
        add_task_datepicker.value = None # DatePicker 값도 초기화
        add_task_calendar_toggle.value = False # 토글 버튼도 초기화 (달력 닫힘)
        display_results(confirmation_message)

def on_complete_task_button_clicked(b): # 이전과 동일
    with out:
        clear_output(wait=True)
        confirmation_message = complete_task_logic(complete_task_number_input.value)
        complete_task_number_input.value = ''
        display_results(confirmation_message)

def on_undo_complete_task_button_clicked(b): # 이전과 동일
    with out:
        clear_output(wait=True)
        confirmation_message = undo_complete_task_logic(undo_complete_task_number_input.value)
        undo_complete_task_number_input.value = ''
        display_results(confirmation_message)

def on_update_task_button_clicked(b):
    with out:
        clear_output(wait=True)
        confirmation_message = update_task_logic(
            update_task_number_input.value,
            update_task_description_input.value,
            update_task_due_date_text.value,    # DatePicker와 연결된 Text 위젯의 값 사용
            update_task_priority_dropdown.value,
            update_task_status_dropdown.value,
            update_task_progress_slider.value
        )
        update_task_number_input.value = ''
        update_task_description_input.value = ''
        update_task_due_date_text.value = '' # 마감일 Text 위젯 초기화
        update_task_datepicker.value = None  # DatePicker 값도 초기화
        update_task_calendar_toggle.value = False # 토글 버튼도 초기화
        display_results(confirmation_message)

def on_delete_task_button_clicked(b): # 이전과 동일
    with out:
        clear_output(wait=True)
        confirmation_message = delete_task_logic(delete_task_number_input.value)
        delete_task_number_input.value = ''
        display_results(confirmation_message)

# 이벤트 핸들러 연결
add_task_button.on_click(on_add_task_button_clicked)
complete_task_button.on_click(on_complete_task_button_clicked)
undo_complete_task_button.on_click(on_undo_complete_task_button_clicked)
update_task_button.on_click(on_update_task_button_clicked)
delete_task_button.on_click(on_delete_task_button_clicked)

# ---- UI 레이아웃 구성 및 최종 표시 ----
initial_load_msg = load_tasks_core()
message_parts = []
if initial_load_msg: message_parts.append(initial_load_msg)
message_parts.append("각 작업을 수행하면 결과와 함께 업데이트된 할 일 목록이 아래에 자동으로 표시됩니다.")
load_message_display.value = f"<p style='color: #555; font-style: italic;'>{'<br>'.join(message_parts)}</p><hr>"

# '새 할 일 추가' 섹션 레이아웃 (마감일 부분 변경)
add_section = widgets.VBox([
    widgets.HTML("<h4>1. 새 할 일 추가</h4>"),
    add_task_description_input,
    add_due_date_control_area, # 수정된 마감일 입력 영역
    add_task_priority_dropdown,
    add_task_button
])

# '할 일 정보 수정' 섹션 레이아웃 (마감일 부분 변경)
update_section_inputs = widgets.VBox([
    update_task_description_input,
    update_due_date_control_area, # 수정된 마감일 입력 영역
    update_task_priority_dropdown,
    update_task_status_dropdown,
    update_task_progress_slider
])
update_section = widgets.VBox([
    widgets.HTML("<h4>2. 선택한 할 일 정보 수정</h4>"),
    update_task_number_input,
    update_section_inputs,
    update_task_button
])

complete_section = widgets.VBox([widgets.HTML("<h4>3. 할 일 완료 표시</h4>"), complete_task_number_input, complete_task_button])
undo_section = widgets.VBox([widgets.HTML("<h4>4. 완료 표시 취소</h4>"), undo_complete_task_number_input, undo_complete_task_button])
delete_section = widgets.VBox([widgets.HTML("<h4>5. 선택한 할 일 삭제</h4>"), delete_task_number_input, delete_task_button])

app_layout = widgets.VBox([
    ui_title, stats_display, load_message_display,
    add_section, widgets.HTML("<hr>"),
    update_section, widgets.HTML("<hr>"),
    complete_section, widgets.HTML("<hr>"),
    undo_section, widgets.HTML("<hr>"),
    delete_section, widgets.HTML("<br><strong>📋 결과 및 목록 (마감일 순 정렬):</strong><br>"),
    out
], layout={'width': '100%', 'max_width': '700px'})

display(app_layout)

# 초기 통계 및 목록 자동 표시
update_stats_display()
with out:
    clear_output()
    initial_task_list_ui = create_task_list_ui()
    display(initial_task_list_ui)